# Feature Engineering

El objetivo es identificar transacciones que evidencian un comportamiento de fraccionamiento(Mala practica transaccional). El set de datos en su forma raw no me provee de columnas que puedan servir de features, por esta razon en función del problema y basandome en el analisis exploratorio de datos voy crear nuevas columnas que me describan la transaccion en terminos de:

* Volumen moneda
* Volumen transaccional
* Tiempo entre transacciones

para cada flujo de moneda y fecha del registro o transacción. 


Con estos datos puedo obtener features que me permitan caracterizar las transacciones en terminos del fraccionamiento transaccional. Segun el concepto es ```fraccionar una transacción en un número mayor de transacciones con menor monto que agrupadas suman el valor de la transacción original. Estas transacciones se caracterizan por estar en una misma ventana de tiempo que suele ser 24 horas y tienen como origen o destino la misma cuenta o cliente. ```

**Nota:** La ventana de 24 horas para efectos de este analisis sera tomada como un dia desde las 00:00:00 hasta las 24:00:00


Tu, profesional de Nequi que esta evaluando mi prueba, te preguntaras probablemente el porque de la decision. Pues bien, en la carpeta pruebas_concepto encontraras un archivo [14h_feature_engineering](pruebas_concepto\03_24hfeature_engineering.ipynb). Alli saque los features contando ventanas de tiempo de 24 entre cada pareja transaccional para contar sus transacciones y tengo dos hallazgos:

* Los intervalos de 24 horas aumentan el espacio muestral de los outliers, sesgando la caracterización del las transacciones que estan en el ultimo quartil. La ventana de 24 horas genera programaticamente un bucle que se detiene cuando hay un espacio de inactividad superior a esas 24 horas, por eso aumenta en desproporcion los outliers. 
* Un usuario que fracciona sus transacciones no suele hacerl0 en una medida de 3-4 transacciones sino probablemente mas de 10 y de forma muy consecutiva. Por lo tanto, así fraccione sus transacciones entre un día y otro se podria detectar las detectaría en cualquier a de lo dos dias por que al menos en uno habrá mas de la mitad de las transacciones totales. 



### Importar librerias

In [2]:
import pandas as pd
import dask.dataframe as dd
from datetime import timedelta
import numpy as np

### Leer la data

In [3]:
route = 'processing_data_0001_dummie.parquet'
df = dd.read_parquet(route)

### Features

*  **Total de transacciones acumuladas por dia**: Esta columna va a llevar el conteo desde un mismo flujo de moneda( account -> subsidiary) en una ventana de 1 dia (= 24 horas) de las transacciones. El proposito de esta columna sera calcular la frecuencia transaccional en cada flujo de moneda durante la ventana de tiempo

In [4]:
def calculate_total_transactions(partition):
    # Ordenar el dataframe por account_number y transaction_date
    partition = partition.sort_values(['transaction_date', 'account_number'])
    
    # Calcular el total de transacciones cada dia y crear una nueva columna 'total_transactions'
    partition['total_transactions'] = partition.groupby([partition['date'], partition['account_number'], partition['subsidiary']]).cumcount() + 1

    return partition


*  **Total en moneda acumulada en por día:** Esta columna va a llevar una suma acumulada de la cantidad de moneda (transaction_amount) objeto de la transacción en la ventana de tiempo. El proposito de esta columna sera calcular el movimiento de moneda en cada flujo. 

In [5]:
def calculate_total_transaction_amount(partition):
    # Ordenar el dataframe por account_number y transaction_date
    partition = partition.sort_values(['transaction_date', 'account_number'])
    
    # Agregar la columna "total_transactions" calculando la suma acumulativa de las transacciones agrupadas por "date", "account_number" y "subsidiary".
    partition['total_transaction_amount'] = partition.groupby([partition['date'], partition['account_number'], partition['subsidiary']])['transaction_amount'].cumsum()

    return partition

* **Diferencia de tiempo en minutos entre cada transaccion:** Esta columna resta el tiempo de la ultima transaccion con la inmediatamente anterior y nos da un tiempo en minutos. El proposito de esta columna es identificar los tiempos entre transacciones en esa ventana de 24 horas

In [6]:
def calculate_time_diff_minutes(partition):
    # Ordenar el dataframe por account_number y transaction_date
    partition = partition.sort_values(['transaction_date', 'account_number'])

    # Calcular la diferencia en tiempo entre transacciones dentro de un mismo account_number y subsidiary
    partition['time_diff_minutes'] = partition.groupby([partition['date'], partition['account_number'], partition['subsidiary']])['transaction_date'].diff()

    # Calcular la diferencia de tiempo en minutos y crear una nueva columna 'time_diff_minutes'
    partition['time_diff_minutes'] = partition['time_diff_minutes'].dt.total_seconds() / 60
    partition['time_diff_minutes'] = partition['time_diff_minutes'].fillna(0)
    
    return partition

*  **Promedio del tiempo en minutos de las transacciones para el flujo de moneda**: Esta columa suma los tiempos entre transacciones para cada flujo de moneda y lo divide por el total de transacciones acumulado en ese registro. El proposito de esta columna es llevar un rate del tiempo entre transacciones anteriores para cada registro

In [7]:
def calculate_avg_time_min_between_transactions(partition):
    # Ordenar el dataframe por account_number y transaction_date
    partition = partition.sort_values(['transaction_date', 'account_number'])

    # Calcular el promedio del tiempo entre transacciones en cada ventana de tiempo (account_number y subsidiary)
    partition['avg_time_min_between_transactions'] = partition.groupby([partition['date'], partition['account_number'], partition['subsidiary']])['time_diff_minutes'].cumsum() / (partition['total_transactions'])

    return partition

In [8]:
# Aplicar cada una de las funciones a cada partición del dataframe
df = df.map_partitions(calculate_total_transactions)
df = df.map_partitions(calculate_total_transaction_amount)
df = df.map_partitions(calculate_time_diff_minutes)
df = df.map_partitions(calculate_avg_time_min_between_transactions)

* **Promedio de moneda en las transacciones durante la ventana de 24 horas:** Esta columna divide el acumulado de moneda para cada flujo de moneda por el total de transacciones dado el registro. El proposito de esta columna es determinar el rate de moneda para cada transaccion segun el historico del flujo de moneda

In [9]:
df['avg_amount_transactions'] = df['total_transaction_amount'] / df['total_transactions']

* **Tiempo total en horas de duracion transaccional:** Esta columna suma el tiempo total en horas del flujo transaccional. Es decir el tiempo en que los usuarios empezaron la primera transaccion hasta que enviaron la ultima. El proposito de esta columna es entender en las transacciones consecutivas cuando tiempo toma el usuario en ejecutarlas. 

In [ ]:
df['total_time_hr'] = ((df['total_transactions'] * df['avg_time_min_between_transactions'])/60).round(2)

In [10]:
# Convertir el dataframe de Dask a pandas para visualizar los resultados
df_featured = df.compute()

In [11]:
df_featured

,merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type,date,total_transactions,total_transaction_amount,time_diff_minutes,avg_time_min_between_transactions,avg_amount_transactions
16066,838a8fa992a4aa2fb5a0cf8b15b63755,4a101c7ee6f4f8f1fcdbd4bc044c59d8,f54e0b6b32831a6307361ed959903e76,2021-01-01 00:00:40,8fee49f3fedc5590551fcb57a2f58a3e,2fa852d1bd38bbbcf4afd629f4a7c51b,5.944,CREDITO,2021-01-01,1,5.944,0.0,0.0,5.944
341948,817d18cd3c31e40e9bff0566baae7758,47d6c9460c5c27d63d6e66d23e598695,6f19909d89f3178ea74b3cee1f20af13,2021-01-01 00:01:08,f75a7c37888408308e02fef9086046fe,516c209606ac9f7e870802c90bffbaf8,59.445,DEBITO,2021-01-01,1,59.445,0.0,0.0,59.445
1097759,817d18cd3c31e40e9bff0566baae7758,8c2df593436163e9cbff2fe3050f0ae1,c1b186a762110afc5d510517b04b6329,2021-01-01 00:01:13,a49b390b90de1454f966c604da9687d3,8c8b5512b0772dc891bef619ecd2acfb,47.556,DEBITO,2021-01-01,1,47.556,0.0,0.0,47.556
124971,838a8fa992a4aa2fb5a0cf8b15b63755,6ed538998f04c804c299dc9fbc4d6e90,dff70ce33784a932ce4a7efc81a43863,2021-01-01 00:01:17,666e5e749c3ae3dea220e023a51c88f4,d0a25a8f11f7228d912e6874a0544d41,5.944,CREDITO,2021-01-01,1,5.944,0.0,0.0,5.944
1401278,838a8fa992a4aa2fb5a0cf8b15b63755,8801042a2192cdb8d96074442e5d4e06,d4b31b123120a4eefd51ba95975f2ae4,2021-01-01 00:01:45,c039cca9581596ee863c3c812835bfff,6b6113f0dfead46eb7f0c21ecb78d567,5.944,DEBITO,2021-01-01,1,5.944,0.0,0.0,5.944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338846,817d18cd3c31e40e9bff0566baae7758,c6f1be268792b5ac30fc0e83da8c2fb8,60f9b595411bb86522c10ec889b5ebc7,2021-11-30 23:58:47,7e59b57bf93138becb4b99d74575e4d5,0ffc883061b99315768f8834a3398c00,35.667,DEBITO,2021-11-30,1,35.667,0.0,0.0,35.667
485842,817d18cd3c31e40e9bff0566baae7758,6920e4ed0e5204a6e95feb6d8b435272,6842d61e41574477f3669e27932fbc8b,2021-11-30 23:59:03,135dc1262ecfa0e28ff41e5e0c31b2e7,a016ae88e28e85164a368b1a3d268be7,118.889,DEBITO,2021-11-30,1,118.889,0.0,0.0,118.889
80710,817d18cd3c31e40e9bff0566baae7758,2872a531e4ce546a996e5590892d3ad1,fcd0f2dad1eae09d05eb002b3f90475d,2021-11-30 23:59:28,8ebb9967f661d5307b8d8d488fdedcba,9e3bc39e3fc648a50cbb084b4dff4baa,59.445,DEBITO,2021-11-30,1,59.445,0.0,0.0,59.445
9934,817d18cd3c31e40e9bff0566baae7758,ee3498c05fa4125e5f81f97578551f7d,da7597b1cba8873e72008fb0346ddeb0,2021-11-30 23:59:33,618d35c18c10b9a1d89ded6b50994d0e,6bd788bbec96d12e5b6a4454ba3389bd,35.667,DEBITO,2021-11-30,1,35.667,0.0,0.0,35.667


# Output feature Engineering

Decido guardar el archivo .parquet con el feauture engineering para usarlo en el futuro sin necesidad de correr todo el proceso dada las limitaciones de computo.

In [12]:
# Guardar el DataFrame procesado en formato Parquet
df_featured.to_parquet('day_featured_data_0002.parquet')